# Проект по SQL

Цель: исследовать базу данных, чтобы сформулировать ценностное предложение для нового продукта.

План:
- Исследование таблиц
- Исследование данных
- Вывод

## Исследование таблиц

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
query = '''SELECT * FROM ratings LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [3]:
query = '''SELECT * FROM reviews LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [4]:
query = '''SELECT * FROM authors LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = '''SELECT * FROM publishers LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Вывод: В бд представленно 5 таблиц с информацией о книгах, авторах, оценках, обзорах и издательствах.

## Исследование данных

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [6]:
query =('''SELECT COUNT(book_id) 
FROM books
WHERE publication_date::date > '2000-01-01'
''')
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


Вывод: Всего в бд представленно 819 книг, вышедших после 1 января 2000 года.

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [7]:
query =('''SELECT b.book_id,
b.title,
COUNT(distinct rev.review_id) AS total_review,
AVG(rat.rating) AS avg_rating
FROM books as b
LEFT JOIN ratings AS rat ON rat.book_id = b.book_id
LEFT JOIN reviews AS rev ON rev.book_id = b.book_id
GROUP BY b.book_id
ORDER BY total_review DESC, avg_rating DESC
''')
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,total_review,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
996,387,Leonardo's Notebooks,0,4.000000
997,221,Essential Tales and Poems,0,4.000000
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


Вывод: Самыми обозреваемыми книгами являются бестселлеры, например Сумерки, Сага о Гарри Поттере.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [8]:
query =('''
SELECT p.publisher_id,
p.publisher,
COUNT(b.book_id) AS total_books
FROM publishers AS p
JOIN books AS b ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id
ORDER BY total_books DESC
LIMIT 1
''')
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher,total_books
0,212,Penguin Books,42


Вывод: Самым большим издательством по кол-ву выпущенных кних является Penguin Books с 42 книгами.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [9]:
query =('''WITH bb AS (
SELECT b.book_id,
b.author_id AS a_id,
COUNT(r.rating_id) AS total_ratings,
AVG(r.rating) AS avg_rating
FROM books AS b
JOIN ratings AS r ON b.book_id = r.book_id
GROUP BY b.book_id
HAVING COUNT(r.rating_id) >= 50
)

SELECT a.author_id,
a.author,
AVG(bb.avg_rating) AS author_avg_rating
FROM authors as a
JOIN bb ON bb.a_id = a.author_id
GROUP BY a.author_id
ORDER BY author_avg_rating DESC
LIMIT 1
''')
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author,author_avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844


Вывод: Самым высокооцененным автором является J.K. Rowling, написавшую успешную серию романов о Гарри Поттере.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [10]:
query =('''WITH users AS(
SELECT rat.username as name,
COUNT(rat.rating_id) as total_ratings
FROM ratings AS rat
GROUP BY rat.username
HAVING COUNT(rat.rating_id) > 48),

u AS(
SELECT rev.username,
COUNT(rev.review_id) AS total_reviews
FROM reviews as rev
JOIN users ON rev.username = users.name
GROUP BY rev.username)

SELECT AVG(total_reviews)
FROM u
''')
pd.io.sql.read_sql(sql=text(query), con = con)


,avg
0,24.0


Вывод: Активные пользователи (которые поставили больше 48 оценок) в среднем пишут 24 обзора на сервисе.

## Вывод

Задача заключалась в исследовании базы данных, чтобы сформулировать ценностное предложение для нового продукта. В БД представленно 5 таблиц с информацией о книгах, авторах, оценках, обзорах и издательствах. Всего в БД представленно 819 книг, вышедших после 1 января 2000 года. Самыми обозреваемыми книгами являются бестселлеры. Самым большим издательством по кол-ву выпущенных кних является Penguin Books. Самым высокооцененным автором является J.K. Rowling. Активные пользователи в среднем пишут 24 обзора на сервисе.